In [1]:
import pandas as pd
import json
import csv
import io
import requests
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import json
from datetime import datetime
import numpy as np

In [2]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202')

In [3]:
jan_21 = pd.read_csv('20220504_all_dec_2020_dropboxes.csv')

In [4]:
may_22 = pd.read_excel('Georgia Drop Box List - 2022 Primary - 5-3-22.xlsx')

In [5]:
jan_21.tail()

,Unnamed: 0,address,lat,long
252,23,"16 W. Forest Ave, Suite 101, Alamo GA 30411",32.148609,-82.782954
253,24,980 Pomce De Leon Ave NE Atlanta HA 30306,33.773877,-84.355291
254,0,"122 W. Water Street, Bainbridge, GA 39817",30.906713,-84.576665
255,1,"32 North Court Ave, Camilla, GA 31730",31.232103,-84.209399
256,2,"46 Old School Rd. Georgetown, GA 39854",31.885024,-85.100646


In [6]:
may_22addr = may_22[['Address']].copy()

In [7]:
may_22addr.Address = may_22addr.Address.str.strip()
may_22addr.columns = may_22addr.columns.str.lower()

In [8]:
may_22addr['may'] = True

In [9]:
addrcompare = pd.merge(jan_21, may_22addr, on = 'address', how = 'outer')

In [10]:
addrcompare[(addrcompare.address.str.contains('Sawnee')) & (pd.isna(addrcompare.address) == False)]

,Unnamed: 0,address,lat,long,may
74,105.0,"1201 Sawnee Drive, Cumming, GA 30040",34.223824,-84.139148,True


In [11]:
processed = addrcompare[(pd.isna(addrcompare.may) == False) & (pd.isna(addrcompare.lat) == False)].copy()

In [12]:
processed1 = processed[['address','lat','long']].copy()

In [13]:
# ran the results by geocodio and ArcGIS. stiching them back together.

In [14]:
geocodio = pd.read_csv('20220505_may_22_dropbox_to_geocode_geocodio_deeb0c3104c781420f452237c143ada5a97788ff.csv')

In [15]:
geocodioft = geocodio[(geocodio['Accuracy Type'] == 'rooftop') | (geocodio['Accuracy Type'] == 'range_interpolation')][['address','Latitude','Longitude']].copy()

In [16]:
geocodioft.columns = ['address','lat','long']

In [17]:
arc = pd.read_excel('c20220505_may_22_dropbox_to__TableToExcel_1.xlsx')

In [18]:
arc1 = arc[['USER_address','X','Y']].copy()
arc1.columns = ['address','long','lat']

In [19]:
weirdo = jan_21[jan_21.address.str.contains('old school', case = False)][['address','lat','long']].copy()

In [20]:
weirdo

,address,lat,long
256,"46 Old School Rd. Georgetown, GA 39854",31.885024,-85.100646


In [21]:
may_22_dropboxes = pd.concat([processed1,weirdo, arc1, geocodioft])

after loading onto arcGIS, we can see that while some addresses were phrased differently they're most likely the same places. using lat long to match 

In [25]:
jan_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  257 non-null    int64  
 1   address     257 non-null    object 
 2   lat         257 non-null    float64
 3   long        257 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 8.2+ KB


In [26]:
may_22_dropboxes.lat = may_22_dropboxes.lat.astype('float')
may_22_dropboxes.long = may_22_dropboxes.long.astype('float')

In [27]:
may_22_dropboxes['in_2022'] = True
jan_21['in_2021'] = True

In [28]:
may_22_dropboxes.lat = [round(x, 6) for x in may_22_dropboxes.lat]
may_22_dropboxes.long = [round(x, 6) for x in may_22_dropboxes.long]

In [29]:
jan_21.lat = [round(x, 6) for x in jan_21.lat]
jan_21.long = [round(x, 6) for x in jan_21.long]

In [30]:
combine = pd.merge(may_22_dropboxes, jan_21, on = ['lat','long'], how = 'outer')

20220506 comparing 2020 addresses from SOS with 2020 addresses from the drop box forms. <br>

In [31]:
import usaddress

In [32]:
formaddr = pd.read_excel('20220505_Drop Box Address Data Cleaned.xlsx','2020')

In [33]:
formaddr.head()

,County,Dropbox,Address,lat,long
0,Appling County,Appling County Administrative Office,"69 TIPPINS ST, BAXLEY GA 31513",NaN,NaN
1,Appling County,Appling County Elections Office,"83 Oak St, Baxley, GA 31513",NaN,NaN
2,Atkinson County,Atkinson County Elections Office,"664 Austin Avenue East, Pearson, GA 31642",NaN,NaN
3,Bacon County,Bacon County Elections Office,"232 West 12th Street, Alma, GA 31510",NaN,NaN
4,Baker County,Baker County Courthouse,"167 Baker Place, Newton, GA 39870",NaN,NaN


In [34]:
formaddr1 = formaddr[['Address']].copy()
formaddr1 = formaddr1.rename(columns = {'Address':'addr_jan_21_form'})

In [35]:
formaddr1.addr_jan_21_form = formaddr1.addr_jan_21_form.str.replace(' GA ',', GA ')

In [36]:
formaddr['parsed'] = [str(usaddress.parse(x)) for x in formaddr.Address]
formaddr.parsed = formaddr.parsed.str.upper()

In [37]:
combine = combine.rename(columns = {'address_x':'address_may_22',
                                   'address_y':'address_jan_21'})

In [38]:
combine['parsed'] = [str(usaddress.parse(x)) if pd.isna(x) == False else x for x in combine.address_jan_21]
combine.parsed = combine.parsed.str.upper()

In [39]:
combine1 = pd.merge(combine, formaddr, on ='parsed', how = 'outer')

In [40]:
combine1[(pd.isna(combine1.address_jan_21) == False) & (pd.isna(combine1.Address) == False)].head()

,address_may_22,lat_x,long_x,in_2022,Unnamed: 0,address_jan_21,in_2021,parsed,County,Dropbox,Address,lat_y,long_y
0,"5238 Evitt St. Ringgold, GA 30736",34.915487,-85.106602,True,22.0,"5238 Evitt St. Ringgold, GA 30736",True,"[('5238', 'ADDRESSNUMBER'), ('EVITT', 'STREETN...",Catoosa County,Ringgold Voting Precinct (Freedom Center),"5238 Evitt St. Ringgold, GA 30736",NaN,NaN
1,"1201 Sawnee Drive, Cumming, GA 30040",34.223824,-84.139148,True,105.0,"1201 Sawnee Drive, Cumming, GA 30040",True,"[('1201', 'ADDRESSNUMBER'), ('SAWNEE', 'STREET...",Forsyth County,Forsyth County Elections Office,"1201 Sawnee Drive, Cumming, GA 30040",NaN,NaN
2,"2808 N Oak St, Valdosta, GA 31602",30.869999,-83.292707,True,194.0,"2808 N Oak St, Valdosta, GA 31602",True,"[('2808', 'ADDRESSNUMBER'), ('N', 'STREETNAMEP...",Lowndes County,Lowndes County Elections Office,"2808 N Oak St, Valdosta, GA 31602",NaN,NaN
3,"56 Short Street, Dahlonega, GA 30533",34.535023,-83.981607,True,195.0,"56 Short Street, Dahlonega, GA 30533",True,"[('56', 'ADDRESSNUMBER'), ('SHORT', 'STREETNAM...",Lumpkin County,Lumpkin County Elections Office,"56 Short Street, Dahlonega, GA 30533",NaN,NaN
4,"167 Baker Place, Newton, GA 39870",31.317829,-84.342261,True,0.0,"167 Baker Place, Newton, GA 39870",True,"[('167', 'ADDRESSNUMBER'), ('BAKER', 'STREETNA...",Baker County,Baker County Courthouse,"167 Baker Place, Newton, GA 39870",NaN,NaN


In [41]:
form1 = combine1[(pd.isna(combine1.address_jan_21) == False) & (pd.isna(combine1.Address) == False)][['Address','lat_x','long_x']].copy()
form1.columns = ['nov_2020_form_address', 'lat','long']

In [42]:
# geocodio results
form2raw = pd.read_csv('20220506_form_address_to_geocode_geocodio_e7cbf25300388ad11b1ee8e30a777d269e8db6fa.csv')

In [43]:
form2 = form2raw[(form2raw['Accuracy Type'] == 'rooftop') | (form2raw['Accuracy Type'] == 'range_interpolation')][['Address','Latitude','Longitude']].copy()
form2.columns = ['nov_2020_form_address','lat','long']

In [44]:
form3raw = pd.read_excel('20220506_form_addr_arc_results1_TableToExcel.xlsx')

In [45]:
form3 = form3raw[form3raw.Score >=90][['address','X','Y']].copy()

In [46]:
form3.columns = ['nov_2020_form_address','long','lat']

In [47]:
form4raw = pd.read_csv('20220506_form_addr_to_fix_geocodio_1d143ee42d222d5fab8ac1b9b34785af034fbc75.csv')

In [48]:
form4 = form4raw[form4raw['Accuracy Score']>= .9][['adjust_address','Latitude','Longitude']].copy()

In [49]:
form4.columns = ['nov_2020_form_address','lat','long']

In [50]:
form5raw = pd.read_excel('c20220506_form_addr_to_arc_r_TableToExcel.xlsx')

In [52]:
form5 = form5raw[['adjust_address','X','Y']].copy()

In [53]:
form5.columns = ['nov_2020_form_address','long','lat']

In [54]:
allforms = pd.concat([form1,form2,form3,form4,form5])

In [55]:
# eyeballing to check if all the dec 2020 were incorporated
formaddr1 = pd.merge(formaddr, allforms, left_on = 'Address', right_on = 'nov_2020_form_address', how = 'outer')

In [56]:
formaddr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   County   297 non-null    object 
 1   Dropbox  297 non-null    object 
 2   Address  297 non-null    object 
 3   lat      0 non-null      float64
 4   long     0 non-null      float64
 5   parsed   297 non-null    object 
dtypes: float64(2), object(4)
memory usage: 14.0+ KB


In [57]:
# adding this address back in
weirdo1 = weirdo.rename(columns = {'address':'nov_2020_form_address'})

In [58]:
allforms = allforms.append(weirdo1)

In [59]:
allforms['in_nov_2020'] = True

In [60]:
allforms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 256
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   nov_2020_form_address  297 non-null    object 
 1   lat                    297 non-null    float64
 2   long                   297 non-null    float64
 3   in_nov_2020            297 non-null    bool   
dtypes: bool(1), float64(2), object(1)
memory usage: 9.6+ KB


In [61]:
# combining the 20220505_dropboxes_unified with allforms
combine = combine.drop(['Unnamed: 0','parsed'], axis = 1)

In [62]:
allforms.lat = [round(x, 6) for x in allforms.lat]
allforms.long = [round(x,6) for x in allforms.long]

In [63]:
allforms = allforms.drop_duplicates()

In [64]:
combine = combine.drop_duplicates()

In [65]:
combine2 = pd.merge(combine, allforms, on = ['lat','long'], how = 'outer')

In [68]:
combine2 = combine2.drop_duplicates()

In [69]:
combine2.to_csv('20220519_jan_21_jan_21_may_22_unified.csv')

Manually strike out the ones eyeballed to be very close, and determined to be the same location geocoded twice because the addresses wee written slightly differently